# Dữ liệu 

In [ ]:
import pandas as pd 
from sklearn.model_selection import train_test_split

In [6]:
# Gọi bộ dữ liệu đã lưu, trong folder /content, đã chia Train/Valid : Test theo tỉ lệ 8:2
path = "../data/cleaned_final"
train_valid_processed_comments = pd.read_csv(f"{path}/train_valid_processed_comments.csv")
test_processed_comments = pd.read_csv(f"{path}/test_processed_comments.csv")

In [7]:
import ast

# Sử dụng hàm literal_eval để chuyển chuỗi thành danh sách
train_valid_processed_comments['Tokenized_Sentence'] = train_valid_processed_comments['Tokenized_Sentence'].apply(ast.literal_eval)
test_processed_comments['Tokenized_Sentence'] = test_processed_comments['Tokenized_Sentence'].apply(ast.literal_eval)

In [8]:
texts = test_processed_comments['Tokenized_Sentence']

# Kiểm tra kiểu phần tử sau khi chuyển đổi
print("Kiểu phần tử:", type(texts.iloc[0]))
print("5 phần tử đầu tiên:", texts.iloc[:5].tolist())

Kiểu phần tử: <class 'list'>
5 phần tử đầu tiên: [['bố', 'khỉ', 'chó', 'kphai', 'người', 'chán'], ['anh', 'tải', 'cái', 'về', 'học', 'thú vị', 'ghê'], ['con', 'nào', 'cũng', 'con', 'con', 'trai', 'tao', 'tập', 'từ', 'nhỏ', 'biết', 'tự', 'lo', 'phụ mẹ', 'cười'], ['gì', 'cũng', 'được', 'yêu', '1', 'người', 'vô tâm', 'đéo', 'mỉm_cười'], ['vãi', 'cả', 'lồn', 'haha', 'càng', 'lên', 'mạng', 'tôi', 'càng', 'thấy', 'số', 'mình', 'tốt', 'cười']]


# Chia train test

In [ ]:
def split_train_valid_test(df_train_valid, df_test, train_size, valid_size, random_state=11):
    X_train, X_valid, y_train, y_valid = train_test_split(
        df_train_valid.drop('Emotion', axis=1),
        df_train_valid['Emotion'],
        test_size=valid_size/(train_size + valid_size),
        random_state=random_state,
        stratify=df_train_valid['Emotion']
    )

    X_test = df_test.drop('Emotion', axis=1)
    y_test = df_test['Emotion']

    return X_train, X_valid, X_test, y_train, y_valid, y_test

,Sentence,Emotion,Tokenized_Sentence,tokens_length
0,cô tiên béo quá . cười_ngượng_ngùng giọng còn ...,Disgust,"[cô, tiên béo, cười_ngượng_ngùng, giọng, còn, ...",87
1,con bé kiểu mày vừa làm cái wtf cười,Other,"[con bé, kiểu, mày, vừa, làm, cái, wtf, cười]",61
2,thân thiết gì loại này cười,Disgust,"[thân thiết, gì, loại, cười]",36
3,áp dụng ngay cười_chảy_nước_mắt,Enjoyment,"[áp dụng, ngay, cười_chảy_nước_mắt]",41
4,nhìn mặt ổng đủ cười chết rồi cười_chảy_nước_mắt,Enjoyment,"[nhìn, mặt, ổng, đủ, cười, chết, cười_chảy_nướ...",66
...,...,...,...,...
8618,"ủa tại sao không nghỉ việc luôn đi , hay lương...",Anger,"[ủa, tại sao, không, nghỉ việc, hay, lương tâm...",86
8619,chuẩn rồi per dù mệt mỏi nhưng vẫn vui cười,Other,"[chuẩn, dù, mệt mỏi, nhưng, vẫn, vui, cười]",57
8620,s tao lại muốn ăn cái này quá làm cho cay vào ...,Enjoyment,"[s, tao, muốn, ăn, cái, làm, cay, vào, chua ng...",144
8621,má phải đọc bình luận mới hiểu được đúng là ng...,Enjoyment,"[má, phải, đọc, bình luận, mới, hiểu, được, đú...",114


# TF-IDF

In [15]:
from sklearn.feature_extraction.text import TfidfVectorizer

def tfidf_embedding(tokens, max_features=8000, min_df=2, max_df=0.95):
    # Chuyển đổi danh sách token thành TF-IDF representation
    # max_features : Số lượng từ vựng tối đa mà  (mặc định: 8000)
    # tokens: Danh sách các token

    # Chuyển danh sách token thành chuỗi văn bản
    texts = [' '.join(token_list) for token_list in tokens]

    # Khởi tạo TfidfVectorizer
    vectorizer = TfidfVectorizer(
        max_features=max_features,
        ngram_range=(1, 2), # Sử dụng unigram và bigram
        min_df=min_df,           # Bỏ qua từ xuất hiện < min_df lần
        max_df=max_df         # Bỏ qua từ xuất hiện > max_df documents
    )

    # fit_transform
    # ma trận TF-IDF (n_samples x n_features)
    tfidf_matrix = vectorizer.fit_transform(texts)

    print("TF-IDF hoàn thành!")
    print(f"Shape: {tfidf_matrix.shape} (số câu x số từ vựng)")
    print("Số từ vựng:", len(vectorizer.get_feature_names_out()))
    print("TF-IDF - Non-zero values:", tfidf_matrix.nnz)

    return tfidf_matrix, vectorizer

def tfidf_transform(tokens, tfidf_vectorizer):
    texts = [' '.join(token_list) for token_list in tokens]
    return tfidf_vectorizer.transform(texts)

In [17]:
# Chuyển sang mảng numpy
def convert_to_array(tfidf_matrix):
    return tfidf_matrix.toarray()

def convert_full_to_array(train_tfidf_matrix, valid_tfidf_matrix, test_tfidf_matrix):
    return convert_to_array(train_tfidf_matrix), convert_to_array(valid_tfidf_matrix), convert_to_array(test_tfidf_matrix)

# Giảm chiều 

Giảm về còn 1 / 3 số chiều 

In [18]:
from sklearn.decomposition import PCA

In [ ]:
def pca(X_train_arr, X_test_arr, X_valid_arr, n_components=1/3):
    pca = PCA(n_components=n_components)
    X_train_pca = pca.fit_transform(X_train_arr)
    X_test_pca = pca.transform(X_test_arr)
    X_valid_pca = pca.transform(X_valid_arr)
    return pca, X_train_pca, X_test_pca, X_valid_pca

In [ ]:
# Import các mô hình phân lớp & hồi quy, cùng các metric đánh giá
import numpy as np
from sklearn.linear_model import LogisticRegression, LinearRegression
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score


# Chuyển về hồi quy 